In [1]:
from decouple import Config, RepositoryEnv
import requests
import pandas as pd
import numpy as np
import json
import os

In [4]:
# Creation d'une classe pour dialoguer plus simplement avec l'API de AT
class APIClient:
    def __init__(self,x_auth_token):
        self.x_auth_token = x_auth_token
        self.base_uri = self.get_api_base_url()
        self.bearer_token = self.get_bearer_token()
        self.session = self.create_session() 

    def get_api_base_url(self):
        return 'https://aides-territoires.beta.gouv.fr/api/'

    def get_bearer_token(self):
        url = 'https://aides-territoires.beta.gouv.fr/api/connexion/'
        headers = {
            'accept': 'application/json',
            'Content-Type': 'application/json',
            'X-AUTH-TOKEN': self.x_auth_token
        }
        response = requests.post(url, headers=headers)
        response.raise_for_status()
        return response.json()['token']

    def create_session(self):
        session = requests.Session()
        session.headers.update({
            'Authorization': f'Bearer {self.bearer_token}',
            'Accept': 'application/json'
        })
        return session

    def simple_request_endpoint(self, endpoint, method='GET'):
        url = f'{self.base_uri}{endpoint}'
        response = self.session.request(method, url)
        response.raise_for_status()
        return response

    def simple_request_url(self, url, method='GET'):
        response = self.session.request(method, url)
        response.raise_for_status()
        return response

In [3]:
config = Config(RepositoryEnv('.env'))
at_x_auth_token = config('X_AUTH_TOKEN')
    
# Utilisation vous avez besoin d'un x_auth_token fournit par aide territoires
client = APIClient(at_x_auth_token)


response = client.simple_request_endpoint('aids/?page=1&itemsPerPage=30')
data = response.json()
data_collect = data['results']
error_counter = 0
while data['next'] or error_counter >10:
    try:
        url = data['next']
        print(url)
        response = client.simple_request_url(url)
        data = response.json()
        data_collect = data_collect + data['results']
        error_counter = 0
    except Exception as error:
        print(error)
        error_counter += 1


https://aides-territoires.beta.gouv.fr/api/aids/?page=2&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=3&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=4&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=5&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=6&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=7&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=8&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=9&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=10&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=11&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=12&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=13&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=14&itemsPerPage=30
https://aides-territoires.beta.gouv.fr/api/aids/?page=15&it

In [6]:
data_at = pd.DataFrame(data_collect)
data_at

,id,slug,url,name,name_initial,short_title,financers,financers_full,instructors,instructors_full,...,recurrence,project_examples,import_data_url,import_data_mention,import_share_licence,date_created,date_updated,project_references,european_aid,is_live
0,162955,appel-a-projets-pedagogiques-culture-cheval-an...,/aides/appel-a-projets-pedagogiques-culture-ch...,Développer des projets pédagogiques en lien av...,Appel à projets pédagogiques Culture Cheval – ...,None,[Conseil départemental de la Manche],"[{'id': 164, 'name': 'Conseil départemental de...",[],[],...,Récurrente,None,None,Ces données sont mises à disposition par le Co...,openlicence20,2024-06-23T06:00:00+00:00,2024-07-07T13:40:29+00:00,[],None,True
1,162956,appel-a-manifestation-dinteret-pour-loccupatio...,/aides/appel-a-manifestation-dinteret-pour-loc...,Candidater à l'appel à manifestation d’intérêt...,Appel à manifestation d’intérêt pour l’occupat...,None,[Conseil régional de Provence-Alpes-Côte d'Azur],"[{'id': 93, 'name': 'Conseil régional de Prove...",[],[],...,Permanente,None,None,Ces données sont mises à disposition par le Co...,openlicence20,2024-06-23T07:00:00+00:00,2024-09-29T07:00:24+00:00,[],None,True
2,162957,passeurs-dimages-en-bourgogne-franche-comte,/aides/passeurs-dimages-en-bourgogne-franche-c...,Mener des projets d'éducation à l'image sur le...,Passeurs d'images,None,[Ministère de la Culture],"[{'id': 96, 'name': 'Ministère de la Culture',...",[],[],...,Ponctuelle,"<section class=""fr-accordion"" data-fr-js-accor...",https://www.culture.gouv.fr,Ces données sont mises à disposition par le Mi...,openlicence20,2024-06-23T07:01:00+00:00,2024-09-15T00:00:13+00:00,[],None,True
3,162965,ecoconception-textile-dhabillement-texhabi,/aides/ecoconception-textile-dhabillement-texh...,Soutenir les projets d’écoconception textile e...,Écoconception textile d'habillement - TEXHABI,None,[ADEME],"[{'id': 22, 'name': 'ADEME', 'logo': 'https://...",[],[],...,Ponctuelle,None,https://agirpourlatransition.ademe.fr/,Ces données sont mises à disposition par l'ADEME.,openlicence20,2024-06-23T08:30:00+00:00,2024-10-13T08:00:20+00:00,[],None,True
4,162966,aqacia-2024-gerer-les-pollutions-a-lozone-et-s...,/aides/aqacia-2024-gerer-les-pollutions-a-lozo...,Gérer les pollutions à l’ozone et sectorielles,AQACIA 2024 - Gérer les pollutions à l’ozone e...,None,[ADEME],"[{'id': 22, 'name': 'ADEME', 'logo': 'https://...",[],[],...,Ponctuelle,None,https://agirpourlatransition.ademe.fr/,Ces données sont mises à disposition par l'ADEME.,openlicence20,2024-06-23T08:30:00+00:00,2024-10-13T08:00:20+00:00,[],None,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4391,138027,11cf-preserver-et-gerer-les-espaces-naturels-s...,/aides/11cf-preserver-et-gerer-les-espaces-nat...,Préserver et gérer les Espaces Naturels Sensib...,"Aide aux études, travaux de restauration écolo...",None,[Conseil départemental de la Haute-Loire],"[{'id': 157, 'name': 'Conseil départemental de...",[],[],...,Permanente,None,None,None,None,2023-04-18T17:58:02+00:00,2024-04-29T14:56:12+00:00,[Restauration écologique / continuité écologique],None,True
4392,111727,7aa1-soutenir-lamenagement-et-lembellissement-...,/aides/7aa1-soutenir-lamenagement-et-lembellis...,"Soutenir la valorisation, l'embellissement et ...",Etudes opérationnelles et aménagement et embel...,None,[Conseil départemental de la Charente],"[{'id': 129, 'name': 'Conseil départemental de...",[],[],...,Permanente,None,None,None,None,2021-12-31T11:36:08+00:00,2024-04-25T15:02:41+00:00,[Végétalisation du cimetière],None,True
4393,117411,cb23-accompagner-les-collectivites-pour-une-ge...,/aides/cb23-accompagner-les-collectivites-pour...,Accompagner les collectivités pour une gestion...,None,None,[ID77],"[{'id': 2019, 'name': 'ID77', 'logo': 'https:/...",[],[],...,Permanente,<ul>\n <li>\n Accompagnement des collectivité...,None,None,None,2022-04-07T17:40:10+00:00,2024-07-30T15:10:55+00:00,[Végétalisation du cimetière],None,Tr

In [8]:
data_at = pd.DataFrame(data_collect)

data_at_select_full = data_at.dropna(subset='description')

data_at_select = data_at_select_full.copy()

sub_min_description = 2000

data_at_select = data_at_select[data_at_select['description'].apply(len)>sub_min_description].reset_index(drop=True)


data_at_select_full.to_csv('hard-database/data_at_select_full.csv',index=False)
data_at_select.to_csv('hard-database/data_at_select.csv',index=False)